In [1]:
# import packages

import io
from datetime import datetime as dt
from PIL import Image
import time
import os

# browser automation
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

In [2]:
# Resource: https://www.youtube.com/watch?v=7KhuEsq-I8o

def extract_img_url(wd, delay, max_images, url):
    def scroll_down(wd):
        
        # # https://stackoverflow.com/questions/12792236/sent-user-to-page-bottom-by-javascript
        # scroll to bottom of the page
        
        # execute JavaScript code
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)

    url = url
    wd.get(url)
    
    # set - prevent duplicate links
    image_urls = set()
    skips = 0
    
    while len(image_urls) + skips < max_images:
        scroll_down(wd)
        # Q4LuWd - class name used by Google
        # locate all elements with the class name
        thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

        for img in thumbnails[len(image_urls) + skips:max_images]:
            
            # click on image, delay (if applicable)
            try:
                img.click()
                time.sleep(delay)
            # image not openable
            except:
                continue
            
            # https://github.com/bijij/ViewImage/issues/268
            # r48jcc - class for google image source
            images = wd.find_elements(By.CLASS_NAME, "r48jcc")

            for image in images:
                
                # get value of the src attribute of image element
                # already exists in set, update parameters 
                if image.get_attribute('src') in image_urls:
                    max_images += 1
                    skips += 1
                    break
                
                # else: add link to set of links if there exists http
                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))

    return image_urls

In [3]:
def download_img(down_path, url, file_name, image_type='JPEG', verbose=True):
    try:

        # image content is url
        img_content = requests.get(url).content
        
        # get the bytes IO of the image
        img_file = io.BytesIO(img_content)    
        # open image using Pillow
        image = Image.open(img_file)
         
        # issue with reading some files
        # https://stackoverflow.com/questions/48248405/cannot-write-mode-rgba-as-jpeg
        if ('png' in url) or ('webp' in url):
#             print(url)
            image = image.convert('RGB')    
        
        # store to file
        file_pth = down_path + file_name
        with open(file_pth, 'wb') as file:
            image.save(file, image_type)

        if verbose == True:
            pass
    
    # error
    except Exception as e:
        print(f'Unable to download image from Google Images due to\n: {str(e)}')
        print(url)

In [34]:
'''
# example link and label
'https://www.google.com/search?q=adidas+yeezy+boost+350+v2+black+red+&tbm=isch&ved=2ahUKEwjaodmN_-GAAxXFt4QIHQbwBXYQ2-cCegQIABAA&oq=adidas+yeezy+boost+350+v2+black+red+&gs_lcp=CgNpbWcQAzIECCMQJzIFCAAQgAQyBAgAEB4yBggAEAgQHjIGCAAQCBAeMgcIABAYEIAEUKwJWKwJYKINaABwAHgAgAGzAYgBkwKSAQMxLjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=Ki7dZNrkN8XvkvQPhuCXsAc&bih=715&biw=1536',
'CP9652'

'''


# search URLS
google_urls = [
    'https://www.google.com/search?sca_esv=557804163&rlz=1C1VDKB_enUS963US963&sxsrf=AB5stBi7V0WNBBgcThzVT72Do_LlsasL5w:1692299335624&q=DZ5485-612+ebay&tbm=isch&source=lnms&sa=X&ved=2ahUKEwj4nYOmsuSAAxW0g4QIHYYLDZQQ0pQJegQIChAB&biw=1536&bih=715&dpr=1.25'
]
    

# directory to save 
labels = [
    'DZ5485-612_ebay',
    
]

In [35]:
# define path for chrome driver
# chrome driver - separate executable that acts as a bridge between 
# Selenium WebDriver and the Google Chrome browser

driver_path = 'C:\\Users\\jeffr\\Desktop\sneaker-classifier-app\\chromedriver_win32'
wd = webdriver.Chrome(executable_path = driver_path)

C:\Users\jeffr\AppData\Local\Temp\ipykernel_31336\1747237004.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path = driver_path)


In [36]:
# TODO
save_dir = 'C:\\Users\\jeffr\\Desktop\sneaker-classifier-app\\data\\unclean\\'

# make directory if not found
for lbl in labels:
    if not os.path.exists(save_dir + lbl):
        # make directory with label name
        os.makedirs(save_dir + lbl)

# get image url links
for url_current, lbl in zip(google_urls, labels):
    urls = extract_img_url(wd, 0, 150, url_current)
    
    # download images
    for i, url in enumerate(urls):
        download_img(save_dir + lbl + "\\", 
                       url=url, file_name=str(i+1)+ '.jpg',verbose=True) 
wd.quit()

Unable to download image from Google Photos due to
: cannot write mode P as JPEG
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3mAyYnNilZRHbAKXSmIRwo8tucMW2RXaTTw&usqp=CAU
